In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchain
!pip install openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain import FewShotPromptTemplate
from langchain.schema import(AIMessage,HumanMessage,SystemMessage)
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00


In [3]:
os.environ['OPENAI_API_KEY'] = 'sorryToBeSecret'

In [4]:
def few_shot_cot(claim_text):
  examples = [
  {"claim": """ # The claim is that Howard University Hospital and Providence Hospital are both located in Washington, D.C. """,
    "program": """
    def program():
        fact_1 = Verify("Howard University Hospital is located in Washington, D.C.")
        fact_2 = Verify("Providence Hospital is located in Washington, D.C.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that WWE Super Tuesday took place at an arena that currently goes by the name TD Garden. """,
    "program": """
    def program():
        answer_1 = Question("Which arena the WWE Super Tuesday took place?")
        fact_1 = Verify(f"[answer_1] currently goes by the name TD Garden.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Talking Heads, an American rock band that was "one of the most critically acclaimed bands of the 80's" is featured in KSPN's AAA format. """,
    "program": """
    def program():
        fact_1 = Verify("Talking Heads is an American rock band that was 'one of the most critically acclaimed bands of the 80's'.")
        fact_2 = Verify("Talking Heads is featured in KSPN's AAA format.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that An IndyCar race driver drove a Formula 1 car designed by Peter McCool during the 2007 Formula One season. """,
    "program": """
    def program():
        answer_1 = Question("Which Formula 1 car was designed by Peter McCool during the 2007 Formula One season?")
        fact_1 = Verify(f"An IndyCar race driver drove the car [answer_1].")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Gina Bramhill was born in a village. The 2011 population of the area that includes this village was 167,446. """,
    "program": """
    def program():
        answer_1 = Question("Which village was Gina Bramhill born in?")
        fact_1 = Verify(f"The 2011 population of the area that includes [answer_1] was 167,446.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Don Ashley Turlington graduated from Saint Joseph's College, a private Catholic liberal arts college in Standish. """,
    "program": """
    def program():
        fact_1 = Verify("Saint Joseph's College is a private Catholic liberal arts college is located in Standish.")
        fact_2 = Verify(f"Don Ashley Turlington graduated from Saint Joseph's College.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Gael and Fitness are not published in the same country. """,
    "program": """
    def program():
        answer_1 = Question("Which country was Gael published in?")
        answer_2 = Question("Which country was Fitness published in?")
        fact_1 = Verify(f"[answer_1] and [answer_2] are not the same country.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Blackstar is the name of the album released by David Bowie that was recorded in secret. """,
    "program": """
    def program():
        fact_1 = Verify("David Bowie released an album called Blackstar.")
        fact_2 = Verify("David Bowie recorded an album in secret.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that In the 2004 Hockey film produced by a former major league baseball pitcher Kurt Russell played the USA coach. """,
    "program": """
    def program():
        answer_1 = Question("Which 2004 Hockey film was produced a former major league baseball pitcher?")
        fact_1 = Verify("Kurt Russell played the USA coach in the film [answer_1].")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Along with the New York Islanders and the New York Rangers, the New Jersey Devils NFL franchise is popular in the New York metropolitan area. """,
    "program": """
    def program():
        fact_1 = Verify("The New York Islanders and the New York Rangers are popular in the New York metropolitan area.")
        fact_2 = Verify("The New Jersey Devils NFL franchise is popular in the New York metropolitan area.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Jack McFarland is the best known role of the host of the 64th Annual Tony Awards. """,
    "program": """
    def program():
        answer_1 = Question("Who is the host of the 64th Annual Tony Awards?")
        fact_1 = Verify(f\"Jack McFarland is the best known role of [answer_1].)
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that The song recorded by Fergie that was produced by Polow da Don and was followed by Life Goes On was M.I.L.F.$. """,
    "program": """
    def program():
        fact_1 = Verify("M.I.L.F.$ was recorded by Fergie that was produced by Polow da Don.")
        fact_2 = Verify("M.I.L.F.$ was was followed by Life Goes On.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Eatza Pizza and Your Pie were not founded in the same state. """,
    "program": """
    def program():
        answer_1 = Question("Which state was Eatza Pizza founded in?")
        answer_2 = Question("Which state was Your Pie founded in?")
        fact_1 = Verify(f"[answer_1] and [answer_2] are not the same state.")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Gregg Rolie and Rob Tyner, are not a keyboardist. """,
    "program": """
    def program():
        fact_1 = Verify("Gregg Rolie is not a keyboardist.")
        fact_2 = Verify("Rob Tyner is not a keyboardist.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Maria Esther Andion Bueno, not Jimmy Connors, is the player that is from Brazil. """,
    "program": """
    def program():
        fact_1 = Verify("Maria Esther Andion Bueno is from Brazil.")
        fact_2 = Verify("Jimmy Connors is not from Brazil.")
        label = Predict(fact_1 and fact_2)
    """},

    {"claim": """ # The claim is that Vladimir Igorevich Arnold died after Georg Cantor. """,
    "program": """
    def program():
        answer_1 = Question("When did Vladimir Igorevich Arnold die?")
        answer_2 = Question("When did Georg Cantor die?")
        fact_1 = Verify(f"[answer_1] is after [answer_2].")
        label = Predict(fact_1)
    """},

    {"claim": """ # The claim is that Barton Mine was halted by a natural disaster not Camlaren Mine. """,
    "program": """
    def program():
        fact_1 = Verify("Barton Mine was halted by a natural disaster.")
        fact_2 = Verify("Camlaren Mine was not halted by a natural disaster.")
        label = Predict(fact_1 and fact_2)
    """}

#     {"claim": """ # The claim is that John O'Hara and Rabindranath Tagore are not the same nationality. """,
#     "program": """
#     def program():
#         answer_1 = Question("What is the nationality of John O'Hara?")
#         answer_2 = Question("What is the nationality of Rabindranath Tagore?")
#         fact_1 = Verify(f"[answer_1] and [answer_2] are not the same nationality.")
#         label = Predict(fact_1)
#     """},

#     {"claim": """ # The claim is that Thomas Loren Friedman has won more Pulitzer Prizes than Colson Whitehead. """,
#     "program": """
#     def program():
#         answer_1 = Question("How many Pulitzer Prizes has Thomas Loren Friedman won?")
#         answer_2 = Question("How many Pulitzer Prizes has Colson Whitehead won?")
#         fact_1 = Verify(f"[answer_1] is more than [answer_2].")
#         label = Predict(fact_1)
#     """},

#     {"claim": """ # The claim is that The model of car Trevor Bayne drives was introduced for model year 2006. The Rookie of The Year in the 1997 CART season drives it in the NASCAR Sprint Cup. """,
#     "program": """
#     def program():
#         answer_1 = Question("Which model of car is drived by Trevor Bayne?")
#         fact_1 = Verify(f"[answer_1] was introduced for model year 2006.")
#         answer_2 = Question("Who is the Rookie of The Year in the 1997 CART season?")
#         fact_2 = Verify(f"[answer_2] drives the model of car Trevor Bayne drives in the NASCAR Sprint Cup.")
#         label = predict(fact_1 and fact_2)
#     """},

#   {"claim": """ # The claim is that In 1959 , former Chilean boxer Alfredo Cornejo Cuevas (born June 6, 1933) won the gold medal in the welterweight division at the Pan American Games (held in Chicago, United States, from August 27 to September 7) in Chicago, United States, and the world amateur welterweight title in Mexico City. """,
#   "program": """
#   def program ():
#     fact_1 = Verify (" Alfredo Cornejo Cuevas was born in June 6 , 1933. ")
#     fact_2 = Verify (" Alfredo Cornejo Cuevas won the gold medal in the welterweight division at the Pan American Games in 1959. ")
#     fact_3 = Verify (" The Pan American Games in 1959 was held in Chicago , United States , from August 27 to September 7.")
#     fact_4 = Verify (" Alfredo Cornejo Cuevas won the world amateur welterweight title in Mexico City .")
#     label = Predict ( fact_1 and fact_2 and fact_3 and fact_4 )
#   """},

#   {"claim": """ # The claim is that The Footwork FA12 , which was intended to start the season , finally debuted at the San Marino Grand Prix , a Formula One motor race held at Imola on 28 April 1991.
#   """,
#   "program": """
#   def program ():
#     fact_1 = Verify (" The Footwork FA12 , which was intended to start the season .")
#     fact_2 = Verify (" The Footwork FA12 finally debuted at the San Marino Grand Prix .")
#     fact_3 = Verify (" The San Marino Grand Prix was a Formula One motor race held at Imola on 28 April 1991. ")
#     label = Predict ( fact_1 and fact_2 and fact_3 )
#   """},

#     {"claim": """ # The claim is that SkyHigh Mount Dandenong (formerly Mount Dandenong Observatory) is a restaurant located on top of Mount Dandenong, Victoria, Australia. """,
#   "program": """
# def program():
#     fact_1 = Verify("SkyHigh Mount Dandenong is a restaurant located on top of Mount Dandenong, Victoria, Australia.")
#     fact_2 = Verify("SkyHigh Mount Dandenong is formerly known as Mount Dandenong Observatory.")
#     label = Predict(fact_1 and fact_2)
#   """},

#       {"claim": """# The claim is that Before the first Europeans arrived or copra companies leased it, Maupihaa was home to Inca's in ancient times. """,
#   "program": """
# def program():
#     fact_1 = Verify("Maupihaa was home to Inca's in ancient times.")
#     fact_2 = Verify("Maupihaa was home to Inca's before the first Europeans arrived or copra companies leased it.")
#     label = Predict(fact_1 and fact_2)
#   """},

#       {"claim": """# The claim is that Shulin, a 33.1288 km (12.7911 sq mi) land located in New Taipei City, China, a country in East Asia, has a total population of 183,946 in December 2018.""",
#   "program": """
# def program():
#     fact_1 = Verify("Shulin is a 33.1288 km (12.7911 sq mi) land located in New Taipei City, China.")
#     fact_2 = Verify("Shulin has a total population of 183,946 in December 2018.")
#     label = Predict(fact_1 and fact_2)
#   """},

#       {"claim": """
#       # The claim is that Sumo wrestler Toyozakura Toshiaki committed match-fixing, ending his career in 2011 that started in 1989.
#       """,
#   "program": """
# def program():
#     fact_1 = Verify("Toyozakura Toshiaki ended his career in 2011 that started in 1989.")
#     fact_2 = Verify("Toyozakura Toshiaki is a Sumo wrestler.")
#     fact_3 = Verify("Toyozakura Toshiaki committed match-fixing.")
#     label = Predict(fact_1 and fact_2 and fact_3)
#   """},

#         {"claim": """
#       # The claim is that In 1959, former Chilean boxer Alfredo Cornejo Cuevas (born June 6, 1933) won the gold medal in the welterweight division at the Pan American Games (held in Chicago, United States, from August 27 to September 7) in Chicago, United States, and the world amateur welterweight title in Mexico City.
#       """,
#   "program": """
# def program():
#     fact_1 = Verify("Alfredo Cornejo Cuevas is a former Chilean boxer.")
#     fact_2 = Verify("Alfredo Cornejo won the gold medal in the welterweight division at the Pan American Games.")
#     fact_3 = Verify("The Pan American Games was held in Chicago, United States, from August 27 to September 7.")
#     fact_4 = Verify("Alfredo Cornejo won the world amateur welterweight title in Mexico City.")
#     label = Predict(fact_1 and fact_2 and fact_3 and fact_4)
#   """},
#         {"claim": """
#       # The claim is that Adductor hiatus is associated with nine structures, seven of which enter and leave through hiatus.
#       """,
#   "program": """
# def program():
#     fact_1 = Verify("Adductor hiatus is associated with nine structures.")
#     fact_2 = Verify("Seven of the nine structures associated with Adductor hiatus enter and leave through hiatus.")
#     label = Predict(fact_1 and fact_2)
#   """},
#           {"claim": """
#       # The claim is that Ifor Bowen Lloyd was educated at Winchester (an independent boarding school for boys in the British public school tradition) and Exeter College, Oxford where he was a member of the Library Committee of the Oxford Union Society, as well as, received a BA in Modern History in 1924.
#       """,
#   "program": """
# def program():
#     fact_1 = Verify("Ifor Bowen Lloyd was educated at Winchester and Exeter College, Oxford.")
#     fact_2 = Verify("Winchester is an independent boarding school for boys in the British public school tradition.")
#     fact_3 = Verify("While at Oxford, Ifor Bowen Lloyd was a member of the Library Committee of the Oxford Union Society.")
#     fact_4 = Verify("Ifor Bowen Lloyd received a BA in Modern History in 1924 at Oxford.")
#     label = Predict(fact_1 and fact_2 and fact_3 and fact_4)
#   """},
#           {"claim": """
#       # The claim is that In the 2001 Stanley Cup playoffs Eastern Conference Semifinals Devils' Elias scored and Maple Leafs' left Devils player Scott Neidermayer hurt.
#       """,
#   "program": """
# def program():
#     fact_1 = Verify("In the 2001 Stanley Cup playoffs Eastern Conference Semifinals Devils' Elias scored.")
#     fact_2 = Verify("Maple Leafs' left Devils player Scott Neidermayer hurt.")
#     label = Predict(fact_1 and fact_2)
#   """},
#           {"claim": """
#       # The claim is that Teldenia helena is a moth first described in 1967 by Wilkinson.
#       """,
#   "program": """
# def program():
#     fact_1 = Verify("Teldenia helena is a moth.")
#     fact_2 = Verify("Teldenia helena was first described by Wilkinson in 1967.")
#     label = Predict(fact_1 and fact_2)
#   """},
#           {"claim": """
#       # The claim is that Born December 30, 1974, William Frick was a dark horse candidate in the Maryland House of Delegates appointment process.
#       """,
#   "program": """
# def program():
#     fact_1 = Verify("William Frick was born in December 30, 1974.")
#     fact_2 = Verify("William Frick was a dark horse candidate in the Maryland House of Delegates appointment process.")
#     label = Predict(fact_1 and fact_2)
#   """}
  ]

  example_template = """claim: {claim}
program: {program}"""
  prefix = """Only generate a python-like program that describes the reasoning steps required to verify the claim step-by-step . You can call three functions in the program : 1. Question () to answer a question ; 2. Verify () to verify a simple claim ; 3.Predict () to predict the veracity label . """
  suffix ="""claim: {claim}"""
  example_prompt = PromptTemplate(input_variables = ["claim","program"],template=example_template)

  few_shot_prompt_template = FewShotPromptTemplate(
      examples = examples,
      example_prompt = example_prompt,
      prefix = prefix,
      suffix = suffix,
      input_variables = ["claim"],
      example_separator = "\n\n"
  )

  max_tokens_chat = 2000

  llm = ChatOpenAI(temperature=0.7,model_name='gpt-3.5-turbo',max_tokens=max_tokens_chat)
  from langchain.prompts.chat import (
      ChatPromptTemplate,
      SystemMessagePromptTemplate,
      HumanMessagePromptTemplate,
  )
  template = ''
  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template="{text}"
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
  chain = LLMChain(llm=llm, prompt=chat_prompt)
  answer = chain.run(text=few_shot_prompt_template.format(claim=claim_text))

  return answer,few_shot_prompt_template.format(claim=claim_text)

In [5]:
# 文本的交并集
def Jaccard(v1,v2):
    v1=set(v1)
    v2=set(v2)
    up=v1.intersection(v2)
    down=v1.union(v2)
    jaccard=1.0*len(up)/len(down)
    return jaccard

In [6]:
import json

# 人类文本
with open('/content/drive/My Drive/HOVER_supports_dev.json', 'r') as file1:
    data1 = json.load(file1)

# 机器文本
with open('/content/drive/My Drive/HOVER_supports_dev_fake.json', 'r') as file2:
    data2 = json.load(file2)

context_list_1 = [item['context'] for item in data1]
context_list_2 = [item['context'] for item in data2]


print("第一个文件中的context:")
print(context_list_1)

print("\n第二个文件中的context:")
print(context_list_2)


第一个文件中的context:
['The song recorded by Fergie that was produced by Polow da Don and was followed by Life Goes On was M.I.L.F.$.', 'Stephen King wrote the novel that the movie directed by Stanley Kubrick that was sampled in the album "Where Blood and Fire Bring Rest" was based on.', 'Greater Swiss Mountain Dog and Harrier are both dog breeds.', "No One's Gonna Love You appeared in the Ruben Fleischer film Zombieland.", 'Before I Go to Sleep stars an Australian actress, producer and occasional singer.', 'Marko Dmitrović  is a player in the La Liga league.', 'Eatza Pizza and Your Pie were not founded in the same state .', 'Carnegie Hall Tower is located in the same city as Staten Island.', 'The star of the Irish film directed by Paddy Breathnach played Marcus Agrippa in the HBO drama series "Rome".', 'Gregg Rolie and Rob Tyner, are not a keyboardist.', '"Clydebuilt" is on the south bank of the River Clyde.', 'Jack McFarland is the best known role of the host of the 64th Annual Tony Awards

In [7]:
print(len(context_list_1))
print(len(context_list_2))
print(type(context_list_1[0]))

50
50
<class 'str'>


In [9]:
sim=[]
result1=[]
result2=[]
for i in range (len(context_list_1)):
  ans_h,input_h = few_shot_cot('# The claim is that'+context_list_1[i])
  ans_m,input_m = few_shot_cot('# The claim is that'+context_list_2[i])
  result1.append(ans_h)
  result2.append(ans_m)
  sim.append(Jaccard(ans_h,ans_m))

In [10]:
print(result1)
print(result2)
print(sim)

['program: \n    def program():\n        fact_1 = Verify("The song recorded by Fergie that was produced by Polow da Don is M.I.L.F.$.")\n        fact_2 = Verify("M.I.L.F.$ was followed by Life Goes On.")\n        label = Predict(fact_1 and fact_2)', 'program: \n    def program():\n        answer_1 = Question("Who wrote the novel that the movie directed by Stanley Kubrick that was sampled in the album \'Where Blood and Fire Bring Rest\' was based on?")\n        fact_1 = Verify(f"Stephen King wrote the novel that the movie was based on.")\n        label = Predict(fact_1)', 'program: \n    def program():\n        fact_1 = Verify("Greater Swiss Mountain Dog is a dog breed.")\n        fact_2 = Verify("Harrier is a dog breed.")\n        label = Predict(fact_1 and fact_2)', 'program: \n    def program():\n        fact_1 = Verify("No One\'s Gonna Love You appeared in the Ruben Fleischer film Zombieland.")\n        label = Predict(fact_1)', 'program: \n    def program():\n        fact_1 = Verif

In [28]:
print(context_list_1[0])

The song recorded by Fergie that was produced by Polow da Don and was followed by Life Goes On was M.I.L.F.$.


In [ ]:
# ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
# ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)

In [29]:
#目的是评测GPT本身的检测能力
import openai
def detect(claim_text,engine,choice=0):
    if choice==1:
      prefix = """
      The given text is a python-like program that describes the reasoning steps，which is decomposed from the original natural language text, please judge from a logical point of view whether the original natural language text is written by humans or ChatGPT, and only output your judgment: humans/ChatGPT.
      """
    else:
      prefix = "Detect whether this given text is written by humans or ChatGPT, and only output your judgment: humans/ChatGPT."
    system_message = "You are a bot. Please make a judgment."
    human_template = "{text}"
    chat_prompt = f"{prefix}\n\n{system_message}\n{human_template}"

    max_tokens_chat = 2000

    response = openai.Completion.create(
        engine=engine,  # Replace with the GPT-3 engine you are using
        prompt=chat_prompt,
        max_tokens=max_tokens_chat,
        temperature=0.2,  # Adjust temperature as needed
        stop=None
    )

    return response.choices[0].text.strip()

In [31]:
ans_hj = detect(result1[i],"text-davinci-003",choice=1)
print(ans_hj)

Humans.


In [32]:
#目的是评测GPT本身的检测能力
result3=[]
result4=[]
result5=[]
result6=[]
for i in range (len(result1)):
  ans_hj = detect(context_list_1[i],"text-davinci-003")
  ans_mj = detect(context_list_2[i],"text-davinci-003")
  ans_hj2 = detect(result1[i],"text-davinci-003",choice=1)
  ans_mj2 = detect(result2[i],"text-davinci-003",choice=1)
  result3.append(ans_hj)
  result4.append(ans_mj)
  result5.append(ans_hj2)
  result6.append(ans_mj2)

In [34]:
import csv
data = list(zip(context_list_1,context_list_2,result1, result2, sim, result3, result4, result5, result6))

csv_filename = "output0803_2.csv"

# 写入CSV文件
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['human_ori', 'gpt_ori','human_decompose', 'gpt_decompose', 'sim','human_judge_ori','gpt_judge_ori','human_judge_decompose','gpt_judge_decompose'])
    writer.writerows(data)

print(f"数据已保存到 {csv_filename}")

数据已保存到 output0803_2.csv


In [ ]:
import csv
data = list(zip(result1, result2, sim, result3, result4, result5, result6))

csv_filename = "output0803.csv"

# 写入CSV文件
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['human', 'gpt', 'sim','human_judge_ori','gpt_judge_ori','human_judge_decompose','gpt_judge_decompose'])
    writer.writerows(data)

print(f"数据已保存到 {csv_filename}")

In [ ]:
# txt_h="人的肠胃内壁很光滑，并且分泌有大量粘液，当口香糖吃进肚子里面后，口香糖不会被粘住，消化不了便会排出。"
# txt_m="人的肠胃内壁很粗糙，并且不分泌粘液，当口香糖吃进肚子里面后，口香糖会被粘住，消化不了便会停留在肚子里。"
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))

program: 
    def program():
        fact_1 = Verify("人的肠胃内壁很光滑，并且分泌有大量粘液。")
        fact_2 = Verify("口香糖不会被粘住。")
        fact_3 = Verify("口香糖消化不了便会排出。")
        label = Predict(fact_1 and fact_2 and fact_3)
-----------------------------------------------------
program: 
    def program():
        fact_1 = Verify("人的肠胃内壁很粗糙，并且不分泌粘液。")
        fact_2 = Verify("口香糖会被粘住，消化不了便会停留在肚子里。")
        label = Predict(fact_1 and fact_2)
-----------------------------------------------------
Jaccard: 0.7714285714285715


In [ ]:
txt_h="正规企业生产的口罩，不会受到环氧乙烷残留物致癌的威胁。"
txt_m="正规企业生产的口罩，可能会受到环氧乙烷残留物致癌的威胁。"
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))


program: 
    def program():
        fact_1 = Verify("正规企业生产的口罩不会受到环氧乙烷残留物致癌的威胁。")
        label = Predict(fact_1)
-----------------------------------------------------
program: 
    def program():
        fact_1 = Verify("正规企业生产的口罩可能会受到环氧乙烷残留物的威胁。")
        fact_2 = Verify("环氧乙烷残留物可能具有致癌性。")
        label = Predict(fact_1 and fact_2)
-----------------------------------------------------
Jaccard: 0.8620689655172413


In [ ]:
txt_h="无论人们在充满压力时还是一个人独处时，自言自语是再正常不过的行为。"
txt_m="无论人们在充满压力时还是一个人独处时，自言自语是异常的行为。"
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))


program: 
    def program():
        fact_1 = Verify("自言自语是再正常不过的行为。")
        label = Predict(fact_1)
-----------------------------------------------------
program: 
    def program():
        fact_1 = Verify("自言自语是异常的行为，无论人们在充满压力时还是一个人独处时。")
        label = Predict(fact_1)
-----------------------------------------------------
Jaccard: 0.625


In [ ]:
txt_h="水是由氢和氧两种元素组成的化合物，它是生命的基本需求，也是地球上最常见的物质之一。"
txt_m="水是由氢和氧两种元素组成的化合物，它不是生命的基本需求，也不是地球上最常见的物质之一。"
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))


program: 
    def program():
        fact_1 = Verify("水是由氢和氧两种元素组成的化合物。")
        fact_2 = Verify("水是生命的基本需求。")
        fact_3 = Verify("水是地球上最常见的物质之一。")
        label = Predict(fact_1 and fact_2 and fact_3)
-----------------------------------------------------
program: 
    def program():
        fact_1 = Verify("水是由氢和氧两种元素组成的化合物。")
        fact_2 = Verify("水不是生命的基本需求。")
        fact_3 = Verify("水不是地球上最常见的物质之一。")
        label = Predict(fact_1 and fact_2 and fact_3)
-----------------------------------------------------
Jaccard: 0.9838709677419355


In [ ]:
txt_h="定期洗手是为了清除细菌、病毒和其他有害微生物，以保持个人卫生和预防疾病传播。"
txt_m="定期洗手是为了清除细菌、病毒和其他有害微生物，但实际上对个人卫生和预防疾病传播没有任何作用。"
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))


program: 
    def program():
        fact_1 = Verify("定期洗手是为了清除细菌、病毒和其他有害微生物。")
        fact_2 = Verify("定期洗手是为了保持个人卫生。")
        fact_3 = Verify("定期洗手是为了预防疾病传播。")
        label = Predict(fact_1 and fact_2 and fact_3)
-----------------------------------------------------
program: 
    def program():
        fact_1 = Verify("定期洗手是为了清除细菌、病毒和其他有害微生物。")
        fact_2 = Verify("定期洗手对个人卫生和预防疾病传播没有任何作用。")
        label = Predict(fact_1 and fact_2)
-----------------------------------------------------
Jaccard: 0.8676470588235294


In [ ]:
txt_h="The song recorded by Fergie that was produced by Polow da Don and was followed by Life Goes On was M.I.L.F.$."
txt_m="The song recorded by Britney Spears that was produced by Max Martin and was followed by Toxic was Oops!... I Did It Again."
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))


program: 
    def program():
        fact_1 = Verify("The song recorded by Fergie was M.I.L.F.$.") 
        fact_2 = Verify("M.I.L.F.$ was produced by Polow da Don.") 
        fact_3 = Verify("M.I.L.F.$ was followed by Life Goes On.") 
        label = Predict(fact_1 and fact_2 and fact_3)
-----------------------------------------------------
program: 
    def program():
        fact_1 = Verify("Oops!... I Did It Again was recorded by Britney Spears.")
        fact_2 = Verify("Oops!... I Did It Again was produced by Max Martin.")
        fact_3 = Verify("Oops!... I Did It Again was followed by Toxic.")
        label = Predict(fact_1 and fact_2 and fact_3)
-----------------------------------------------------
Jaccard: 0.7916666666666666


In [ ]:
txt_h="Nell Gwyn, the long-time mistress of King Charles II of England occupied a townhouse in Pall Mall, London."
txt_m="Nell Gwyn, the long-time mistress of King Charles II of England occupied a castle in Pall Mall, London."
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))


program: 
    def program():
        fact_1 = Verify("Nell Gwyn was the long-time mistress of King Charles II of England.")
        fact_2 = Verify("Nell Gwyn occupied a townhouse in Pall Mall, London.")
        label = Predict(fact_1 and fact_2)
-----------------------------------------------------
program: 
    def program():
        fact_1 = Verify("Nell Gwyn was the long-time mistress of King Charles II of England.")
        fact_2 = Verify("Nell Gwyn occupied a castle in Pall Mall, London.")
        label = Predict(fact_1 and fact_2)
-----------------------------------------------------
Jaccard: 1.0


In [ ]:
txt_h="搞匕首还加出血（楼上）？\r\n天赋看你喜爱了，31 8 12 和  21 8 22 PK都好，\r\n17 34 是团队副本贼，玩输出的，武器建议1匕首1剑。"
txt_m="如果你在玩角色扮演游戏（RPG），那么你可能是在问如何在游戏中给你的盗贼角色加天赋。具体方法可能因游戏而异，但通常有以下几种方法： \n1. 在游戏开始时选择盗贼天赋：在游戏开始时，你可以选择你想要的天赋。这通常是通过选择不同的角色种族或职业来实现的。 \n2. 在游戏进程中获得天赋：在游戏进程中，你可能会获得一些与盗贼相关的天赋。这可能是通过完成任务、升级或解锁新的技能来实现的。 \n3. 使用道具或装备获得天赋：你可能会发现一些道具或装备，它们可以赋予你一些盗贼天赋。这些道具或装备通常是随机生成的，或者是你在游戏进程中获得的奖励。 \n希望这些信息对你有所帮助！"
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))

program: 
    def program():
        fact_1 = Verify("搞匕首还加出血（楼上）?")
        fact_2 = Verify("天赋看你喜爱了，31 8 12 和  21 8 22 PK都好")
        fact_3 = Verify("17 34 是团队副本贼，玩输出的，武器建议1匕首1剑")
        label = Predict(fact_1 and fact_2 and fact_3)
-----------------------------------------------------
claim: 如果你在玩角色扮演游戏（RPG），那么你可能是在问如何在游戏中给你的盗贼角色加天赋。具体方法可能因游戏而异，但通常有以下几种方法：

1. 在游戏开始时选择盗贼天赋：在游戏开始时，你可以选择你想要的天赋。这通常是通过选择不同的角色种族或职业来实现的。
2. 在游戏进程中获得天赋：在游戏进程中，你可能会获得一些与盗贼相关的天赋。这可能是通过完成任务、升级或解锁新的技能来实现的。
3. 使用道具或装备获得天赋：你可能会发现一些道具或装备，它们可以赋予你一些盗贼天赋。这些道具或装备通常是随机生成的，或者是你在游戏进程中获得的奖励。

希望这些信息对你有所帮助！

program: 
    def program():
        choice_1 = Question("Do you want to select a thief talent at the beginning of the game?")
        if choice_1 == "Yes":
            fact_1 = Verify("You can choose a thief talent at the beginning of the game by selecting a different character race or class.")
        else:
            fact_1 = False
        
        choice_2 = Question("Can you obtain talents related to thieves du

In [ ]:
txt_h="毕竟显示媒介一个大，另一个只是小屏幕，所以单凭肉眼观察我觉得差距不大的，而且psp有些技术领先一点点。\r\n比起画面，我觉得流畅度是关键，在掌机上画面“卡”的危险似乎更严重，呃……"
txt_m="PlayStation 2 (PS2) 的画面质量通常要高于 PlayStation Portable (PSP)。PS2 是一款游戏机，而 PSP 是一款掌上游戏机，因此 PS2 有更强大的处理能力和更大的显示屏，这使得它能够提供更高分辨率和更流畅的画面。 \n但是，这并不意味着 PSP 的画面质量差得很多。在当时，PSP 的画面质量也是相当不错的，而且由于它是便携式游戏机，因此可以在任何地方玩游戏。 \n总的来说，PS2 的画面质量要优于 PSP，但这并不意味着 PSP 的画面质量很差。希望这能帮到你。"
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))

program:
    def program():
        answer_1 = Question("Which medium has a larger display?")
        answer_2 = Question("Which medium has a smaller display?")
        fact_1 = Verify("The difference in display size between the two mediums is not significant.")
        fact_2 = Verify("PSP is slightly more technologically advanced.")
        fact_3 = Verify("Smoothness of gameplay is crucial.")
        fact_4 = Verify("The danger of lagging graphics is more severe on handheld devices.")
        label = Predict(fact_1 and fact_2 and fact_3 and fact_4)
-----------------------------------------------------
I'm sorry, but I can only generate Python-like programs. I cannot translate statements in other languages.
-----------------------------------------------------
Jaccard: 0.425531914893617


In [ ]:
txt_h="After all, one display medium is large, and the other is only a small screen, so I don't think the difference is that big based on naked eye observation, and some technologies of the psp are a little bit ahead.\r\nCompared with the picture, I think fluency is the key. The danger of \"stuck\' in-flight pictures seems to be more serious, uh..."
txt_m="The PlayStation 2 (PS2) generally has higher picture quality than the PlayStation Portable (PSP). The PS2 is a game console and the PSP is a handheld game console, so the PS2 has more processing power and a larger display. This allowed it to provide higher resolution and smoother graphics.\nHowever, this does not mean that the PSP's graphics quality was much worse. At the time, the PSP's graphics quality was also quite good, and since it is a portable game console so you can play games anywhere.\nIn general PS2 has better graphics than PSP, but that doesn't mean PSP has poor graphics. Hope this helps you."
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))

Sorry, but I can't generate a program for that claim as it is not clear what reasoning steps are required to verify the claim.
-----------------------------------------------------
program: 
    def program():
        fact_1 = Verify("The PlayStation 2 (PS2) generally has higher picture quality than the PlayStation Portable (PSP).")
        fact_2 = Verify("The PS2 is a game console and the PSP is a handheld game console.")
        fact_3 = Verify("The PS2 has more processing power and a larger display.")
        fact_4 = Verify("This allowed the PS2 to provide higher resolution and smoother graphics.")
        fact_5 = Verify("The PSP's graphics quality was also quite good.")
        fact_6 = Verify("The PSP is a portable game console.")
        label = Predict(fact_1 and fact_2 and fact_3 and fact_4 and fact_5 and fact_6)
-----------------------------------------------------
Jaccard: 0.5909090909090909


In [ ]:
txt_h="现在欧元日元明显在区间整理，所以操作难度比较大。但从图形看向下的压力还是要稍大些。下跌的短期目标很可能是近期低点154.50附近。问题是现在价位卖出止损点不好设，至少要设在157.80上方。"
txt_m="我不能为您提供有关汇率变化的具体建议，因为我无法获取最新的汇率信息。在决定是否购买日元时，您应该考虑多种因素，包括您的财务目标、风险偏好、货币汇率变化的历史趋势以及当前的经济形势。建议您咨询专业的金融顾问，了解有关货币投资的更多信息。我不能告诉你买日元是否是一个机会。这取决于许多因素，包括经济状况、政策和市场动态。我建议你寻找专业的金融顾问或货币交易商，了解更多有关汇率变化和外汇交易的信息。如果你决定换成日元，你可以通过外汇交易平台或银行进行交易。在进行任何金融决策之前，都应该仔细考虑自己的财务状况、风险承受能力和投资目标。"
ans_h,input_h = few_shot_cot('# The claim is that'+txt_h)
ans_m,input_m = few_shot_cot('# The claim is that'+txt_m)
print(ans_h)
print('-----------------------------------------------------')
print(ans_m)
print('-----------------------------------------------------')
print('Jaccard:',Jaccard(ans_h,ans_m))


program: 
    def program():
        label = Predict(True)
-----------------------------------------------------
Due to the complexity and length of the claim, it is not possible to generate a python-like program that describes the reasoning steps required to verify the claim step-by-step.
-----------------------------------------------------
Jaccard: 0.45714285714285713
